# Name : Samer AlTaki
### IOT & Computer Vision 
## Task4 : Face Mask Detection in Real Time

  #  

## 1.1Face Mask Detection in Real Time

In [5]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [3]:
# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])



from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('train',target_size=(150,150),batch_size=16 ,class_mode='binary')

test_set = test_datagen.flow_from_directory('test',target_size=(150,150),batch_size=16,class_mode='binary')

model_saved=model.fit_generator(training_set,epochs=10,validation_data=test_set)

model.save('FMmodel.h5',model_saved)

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


<ipython-input-3-149eee34bb26>:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(training_set,epochs=10,validation_data=test_set)


Epoch 1/10
83/83 [==============================] - 101s 1s/step - loss: 0.4050 - accuracy: 0.8204 - val_loss: 0.1660 - val_accuracy: 0.9485
Epoch 2/10
83/83 [==============================] - 92s 1s/step - loss: 0.1554 - accuracy: 0.9346 - val_loss: 0.1043 - val_accuracy: 0.9691
Epoch 3/10
83/83 [==============================] - 88s 1s/step - loss: 0.1205 - accuracy: 0.9574 - val_loss: 0.1307 - val_accuracy: 0.9381
Epoch 4/10
83/83 [==============================] - 90s 1s/step - loss: 0.1202 - accuracy: 0.9543 - val_loss: 0.1298 - val_accuracy: 0.9691
Epoch 5/10
83/83 [==============================] - 93s 1s/step - loss: 0.1302 - accuracy: 0.9498 - val_loss: 0.0909 - val_accuracy: 0.9536
Epoch 6/10
83/83 [==============================] - 89s 1s/step - loss: 0.0774 - accuracy: 0.9741 - val_loss: 0.0769 - val_accuracy: 0.9691
Epoch 7/10
83/83 [==============================] - 92s 1s/step - loss: 0.0741 - accuracy: 0.9756 - val_loss: 0.0701 - val_accuracy: 0.9639
Epoch 8/10
83/83 [=

In [6]:
# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel=load_model('FMmodel.h5')

#cap=cv2.VideoCapture(0)
cap = cv2.VideoCapture('test_video.mp4')

face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()